In [1]:
!pip install tqdm

In [1]:
!pip install groq

In [2]:
import pandas as pd
import os
from tqdm import tqdm
from huggingface_hub import InferenceClient
from google.colab import files

In [3]:
os.environ["GROQ_API_KEY"] = "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"

In [4]:
from groq import Groq
import numpy as np

In [5]:
client = Groq(api_key="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")

In [6]:
def predict_case_outcome(case_text, client):
    prompt = f"""System: You are a precise legal predictor with specialized training in court case outcome analysis. Accuracy is critical. The user will provide a legal case text, and you must predict who wins.

User: Analyze this legal case and predict the outcome:

{case_text}

Follow this structured analysis approach:
1. First, identify the exact legal question(s) before the court
2. List key facts that directly impact the legal determination
3. Examine controlling statutes and precedent cases mentioned
4. Compare the strength of each party's arguments and evidence
5. Consider procedural and jurisdictional factors that may influence the outcome

After your analysis, predict the outcome by stating ONLY a single digit:
0 = Plaintiff wins
1 = Defendant wins

Your response should contain ONLY the digit 0 or 1, with no explanation or additional text."""
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are a legal reasoning expert."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=5
        )
        output = response.choices[0].message.content.strip()

        if "0" in output:
            return 0
        elif "1" in output:
            return 1
        else:
            print(f"Warning: Unexpected model response: {output}. Defaulting to 1.")
            return 1

    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        return 1

# 3. Function to process cases from CSV
def process_cases(input_file, output_file):
    if not os.path.exists(input_file):
        print(f"Please upload the {input_file} file")
        uploaded = files.upload()
        if input_file not in uploaded:
            raise ValueError(f"Failed to upload {input_file}")

    df = pd.read_csv(input_file)

    if 'id' not in df.columns or 'text' not in df.columns:
        raise ValueError("Input CSV must contain 'id' and 'text' columns")

    results = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing cases"):
        case_id = row['id']
        case_text = row['text']

        outcome = predict_case_outcome(case_text, client)
        results.append({'id': case_id, 'value': outcome})

    output_df = pd.DataFrame(results)
    output_df.to_csv(output_file, index=False)
    print(f"✅ Predictions saved to {output_file}")

In [7]:
import pandas as pd

# Load the original CSV file
input_file = 'cases.csv'
output_file = 'cases1.csv'
df = pd.read_csv(input_file)
df_subset = df.head(50)
df_subset.to_csv(output_file, index=False)
print(f"First 50 entries saved to {output_file}")


First 50 entries saved to cases1.csv


In [8]:
if __name__ == "__main__":
  input_csv = "/content/cases1.csv"
  output_csv = "/content/output3.csv"
  process_cases(input_csv, output_csv)

Processing cases:  20%|██        | 10/50 [03:21<10:07, 15.20s/it]

Error in prediction: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.3-70b-versatile` in organization `org_01js7pf3ygfghvttvdrxb3gfww` service tier `on_demand` on tokens per minute (TPM): Limit 12000, Requested 18173, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Processing cases:  50%|█████     | 25/50 [07:19<03:42,  8.89s/it]

Error in prediction: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7pf3ygfghvttvdrxb3gfww` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98722, Requested 2390. Please try again in 16m0.623999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error in prediction: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7pf3ygfghvttvdrxb3gfww` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98722, Requested 10548. Please try again in 2h13m29.001999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Processing cases:  52%|█████▏    | 26/50 [07:19<02:30,  6.26s/it]

...

Error in prediction: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7pf3ygfghvttvdrxb3gfww` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99573, Requested 6768. Please try again in 1h31m18.458s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error in prediction: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7pf3ygfghvttvdrxb3gfww` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99573, Requested 2337. Please try again in 27m29.957999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Processing cases:  98%|█████████▊| 49/50 [07:26<00:00,  7.91it/s]

Error in prediction: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7pf3ygfghvttvdrxb3gfww` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99573, Requested 1109. Please try again in 9m48.831s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error in prediction: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7pf3ygfghvttvdrxb3gfww` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99573, Requested 4345. Please try again in 56m24.620999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Processing cases: 100%|██████████| 50/50 [07:26<00:00,  8.93s/it]

Error in prediction: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7pf3ygfghvttvdrxb3gfww` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99573, Requested 5946. Please try again in 1h19m27.764999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
✅ Predictions saved to /content/output3.csv
